In [1]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import pymupdf4llm

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
# from llama_index.core import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

from langchain.retrievers import EnsembleRetriever, ContextualCompressionRetriever
from langchain_community.retrievers import BM25Retriever
# from langchain_teddynote.retrievers import KiwiBM25Retriever

from langchain.retrievers.document_compressors import FlashrankRerank

/home/psm/anaconda3/envs/anz/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

In [3]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [5]:
def process_pdf_by_markdown(file_path, chunk_size=512, chunk_overlap=32):
    md_text = pymupdf4llm.to_markdown(file_path)
    
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    splits = text_splitter.split_documents(md_chunks)
    return splits

def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-large"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        
        # Markdown 형식으로 처리해서 벡터 DB 생성
        chunks = process_pdf_by_markdown(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr",
                                    search_kwargs={'k': 5, 'fetch_k': 10})
        
        bm25_retriever = BM25Retriever.from_documents(chunks, preprocess_func=kiwi_tokenize,
                                                      search_kwargs={'k': 5, 'fetch_k': 10})

        # initialize the ensemble retriever
        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5],
        )

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': ensemble_retriever
        }
    return pdf_databases

def documents_from_pdfs(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    documents = []
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        documents.extend(chunks)
    return documents


In [6]:
base_directory = '/home/psm/model/data' # Your Base Directory
df = pd.read_csv('/home/psm/model/data/test.csv')
# documents = documents_from_pdfs(df, base_directory)
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


/home/psm/anaconda3/envs/anz/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:  11%|█         | 1/9 [00:07<00:59,  7.40s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:12<00:41,  5.93s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:44<00:13,  6.99s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:50<00:06,  6.72s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:57<00:00,  6.36s/it]


In [6]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [7]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

In [15]:
q_n = 10
query = df['Question'][q_n]
# answer = df['Answer'][q_n]
source = df['Source'][q_n]

normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
retriever = normalized_keys[normalize_string(source)]['retriever']

print(source)
print(query)
# print(answer)
print()

docs = retriever.invoke(query)
pretty_print_docs(docs)

보건복지부_부모급여(영아수당) 지원
부모급여(영아수당)의 2024년 확정된 예산은 몇백만원인가요?

Document 1:

|---|---|---|---|---|---|---|
|사업명|2022년 결산|2023년 예산 본예산(A)|2024년||증감 (B-A) (B-A)/A||
||||정부안|확정(B)||(B-A)/A|
|부모급여(영아수당) 지원|341,751|1,621,454|2,888,694|2,888,694|1,267,240|78.2|  
2022년 2023년 예산 2024년 증감
사업명  
결산 본예산(A) (B-A)  
정부안 확정(B) (B-A)/A  
부모급여(영아수당) 지원 341,751 1,621,454 2,888,694 2,888,694 1,267,240 78.2  
4. 사업목적·내용  
(단위: 백만원, %)  
ㅇ (부모급여 지원) 출산 및 양육으로 손실되는 소득을 보전하고, 주 양육자의 직접돌봄이  
중요한 아동발달의 특성에 따라 영아기 돌봄을 두텁게 지원하기 위해 부모급여 지급  
-  영아수당을부모급여로변경‧확대  
‧  
-----
----------------------------------------------------------------------------------------------------
Document 2:

# 5. 사업근거 및 추진경위  
① 법령상 근거 및 조항 적시 : 아동수당법 제4조제5항  
◈아동수당법제4조(’23.9.14. 시행)
제4조(아동수당의지급대상및지급액) ①아동수당은8세미만의아동에게매월10만원을지급한다.  
⑤제1항에도불구하고만2세미만아동에게매월50만원이상으로서대통령령으로정
하는금액을추가로지급한다.  
부칙제6조(아동수당추가지급금액에관한특례) 제4조제5항의개정규정에도불구하고20  
23년12월31일까지는30만원이상의금액중에서보건복지부장관이별도로정하여
고시하는금액을지급한다.  
② 추진경위  
제  
‧  
｣ 5  
-  예비타당성조사 통과(’21.8.), 근거법 마련(’21.12.) 

In [8]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID
    model_id = "rtzr/ko-gemma-2-9b-it"
    # model_id = "/home/psm/model/resource/notebook/results/checkpoint-500"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        # temperature=0.2,
        return_full_text=False,
        max_new_tokens=300,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf, tokenizer

In [9]:
# LLM 파이프라인
llm, tokenizer = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]
/home/psm/anaconda3/envs/anz/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [10]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

model = HuggingFaceCrossEncoder(model_name="Dongjin-kr/ko-reranker")
compressor = CrossEncoderReranker(model=model, top_n=5)

In [11]:
print(type(compressor))

<class 'langchain.retrievers.document_compressors.cross_encoder_rerank.CrossEncoderReranker'>


In [16]:



# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']
    
    # Compressed revriever
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever,
    )

    # RAG 체인 구성
    template = """
    ### 다음 정보를 바탕으로 질문에 답하세요 (간결하게 답변하고 똑같은 단어는 반복하지 마세요):
    {context}

    ### 질문: {question}

    ### 답변:
    """
    prompt = PromptTemplate.from_template(template)
    
    rag_chain = (
        {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })
    
    


Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:01<02:28,  1.53s/it]

Answer: 2,300,000 백만원




Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   1%|          | 1/98 [00:03<05:14,  3.25s/it]


KeyboardInterrupt: 

In [11]:
new_data = {
    'SAMPLE_ID': ['TRAIN_{:03d}'.format(i) for i in range(len(results))],
    'Answer': ""
}
submit_df = pd.DataFrame(new_data)
submit_df

,SAMPLE_ID,Answer
0,TRAIN_000,
1,TRAIN_001,
2,TRAIN_002,
3,TRAIN_003,
4,TRAIN_004,
...,...,...
95,TRAIN_095,
96,TRAIN_096,
97,TRAIN_097,
98,TRAIN_098,


In [13]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("/home/psm/model/data/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'].split("### 질문")[0].strip() for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]


# 결과를 CSV 파일로 저장
submit_df.to_csv("/home/psm/model/data/08164_submission.csv", encoding='UTF-8-sig', index=False)

In [ ]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [21]:
# 제출용 샘플 파일 로드
# submit_df = pd.read_csv("/home/psm/model/data/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("/home/psm/model/data/0816_train_infer_1.csv", encoding='UTF-8-sig', index=False)